In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import pickle
import json

# Carrega os arquivos de dicionários e modelo

In [2]:
def carregaArquivos():
    #Carrega dicionário label
    labelencoder_dictArquivo = open('labelencoder_dict.sav', 'rb')
    labelencoder_dict = pickle.load(labelencoder_dictArquivo)
    
    #Carrega dicionário onehotencoder
    onehotencoder_dictArquivo = open('onehotencoder_dict.sav', 'rb')
    onehotencoder_dict = pickle.load(onehotencoder_dictArquivo)
    
    #Carrega modelo
    modelo_arquivo = open('modeloTreinadoMoveMe.sav', 'rb')
    modelo_carregado = pickle.load(modelo_arquivo)
    return labelencoder_dict, onehotencoder_dict, modelo_carregado

# Recebe arquivos carregados

In [3]:
label_dict, onehot_dict, modelo = carregaArquivos()

C:\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Função que compara dados com o dicionário

In [4]:
def getEncoded(test_data,labelencoder_dict,onehotencoder_dict):
    test_encoded_x = None
    for i in range(0,test_data.shape[1]):
        label_encoder =  labelencoder_dict[i]
        feature = label_encoder.transform(test_data.iloc[:,i])
        feature = feature.reshape(test_data.shape[0], 1)
        onehot_encoder = onehotencoder_dict[i]
        feature = onehot_encoder.transform(feature)
        if test_encoded_x is None:
              test_encoded_x = feature
        else:
              test_encoded_x = np.concatenate((test_encoded_x, feature), axis=1)
    return test_encoded_x

# Escolha de restaurante do usuário

In [5]:
escolha = ['Japanese', 'Excellent', 4, 600, 4]

# Classifica escolha do usuário

In [6]:
def classificaEscolha(escolhaLista, label_dict, onehot_dict, modelo_carregado):
    #Converte valores string em dataframe
    colunasCategoricasPredicao = [{'cozinha': escolhaLista[0], 'votos': escolhaLista[1]}]
    colunasCategoricasPredicaoDic = pd.DataFrame(colunasCategoricasPredicao)
    
    #Busca dataframe no dicionário
    dicionarioCategoriasPredicaoParte1 = getEncoded(colunasCategoricasPredicaoDic, label_dict, onehot_dict)
    
    #Transforma partes em numpy array
    PegandoParte2Escolha = np.asarray(escolhaLista[2:])
    Parte2Escolha = PegandoParte2Escolha.reshape(1,3)
    escolha_para_predicao = np.concatenate((dicionarioCategoriasPredicaoParte1, Parte2Escolha), axis=1)
    return modelo.predict(escolha_para_predicao)

In [7]:
print("Média de preço predição carregada para características buscadas: ", classificaEscolha(escolha, label_dict, onehot_dict, modelo))

Média de preço predição carregada para características buscadas:  [230]


In [8]:
from flask import Flask, abort, jsonify, request

In [14]:
url = "http://127.0.0.1:8081/"
dados = json.dumps({'cozinha': escolha[0], 'taxa_votos':escolha[1], 'alcance_preco':escolha[2],'classifi_agregada':escolha[3],'votos':escolha[4]})
r = request.post(url, dados)

print(r.json())

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.